# [Agents] (https://python.langchain.com/v0.2/docs/tutorials/agents/)  
this notebook explores builing an agent using LangChain

In [11]:
from dotenv import load_dotenv; load_dotenv()
open_ai_model = "gpt-3.5-turbo-0125"


In [3]:
# we are using Tavily, a search engine tool specially designed for llms
# tavily requires an API key, make sure it is named TAVILY_API_KEY
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)


[{'url': 'https://weatherspark.com/h/m/557/2024/1/Historical-Weather-in-January-2024-in-San-Francisco-California-United-States', 'content': 'San Francisco Temperature History January 2024\nHourly Temperature in January 2024 in San Francisco\nCompare San Francisco to another city:\nCloud Cover in January 2024 in San Francisco\nDaily Precipitation in January 2024 in San Francisco\nObserved Weather in January 2024 in San Francisco\nHours of Daylight and Twilight in January 2024 in San Francisco\nSunrise & Sunset with Twilight in January 2024 in San Francisco\nSolar Elevation and Azimuth in January 2024 in San Francisco\nMoon Rise, Set & Phases in January 2024 in San Francisco\nHumidity Comfort Levels in January 2024 in San Francisco\nWind Speed in January 2024 in San Francisco\nHourly Wind Speed in January 2024 in San Francisco\nHourly Wind Direction in 2024 in San Francisco\nAtmospheric Pressure in January 2024 in San Francisco\nData Sources\n See all nearby weather stations\nLatest Repo

In [12]:
#creating model 
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model=open_ai_model)



In [14]:
# below is an example of how we can invoke the model to get a response
# it will send a query to the openai apiand respond with an AIMessage 
pass
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content



'Hello! How can I assist you today?'

## introducing tools

In [17]:
# there are formats tools can be in: dictionary desribing use, inheriting form a variety of basetool classes
tools = [search]

# we then use langgraph to build a react agent (reason, action)
# we pass it the unmodified model and a list of tools 
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)



In [19]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]



[HumanMessage(content='whats the weather in sf?', id='5a387348-c53b-45c5-9342-88829b50667b'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1s1bG62x6JqUZ5oRF0PY7B20', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 88, 'total_tokens': 109}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-90b386ab-398b-47d4-84b5-0ccbe1a73d6b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_1s1bG62x6JqUZ5oRF0PY7B20'}], usage_metadata={'input_tokens': 88, 'output_tokens': 21, 'total_tokens': 109}),
 ToolMessage(content='[{"url": "https://weatherspark.com/h/m/557/2024/1/Historical-Weather-in-January-2024-in-San-Francisco-California-United-States", "content": "San Francisco Temperature History

In [26]:
# response is a dict with one item, message
# message is the full process of the conversations: human message, ai messages, tool use documentation
# the last message should include the final message (in a AIMessage form)
response['messages'][-1].content

'The current weather in San Francisco is as follows:\n- Temperature: 64.0°F (17.8°C)\n- Condition: Sunny\n- Wind: 6.1 km/h from WSW\n- Humidity: 75%\n- Cloud Cover: 0%\n- Visibility: 16.0 km\n\nFor more details, you can visit [Weather API](https://www.weatherapi.com/).'

In [28]:
print(response['messages'][-1].content)

The current weather in San Francisco is as follows:
- Temperature: 64.0°F (17.8°C)
- Condition: Sunny
- Wind: 6.1 km/h from WSW
- Humidity: 75%
- Cloud Cover: 0%
- Visibility: 16.0 km

For more details, you can visit [Weather API](https://www.weatherapi.com/).


In [33]:
# this is a different way to track what the agent is doing, it prints out each step it takes
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")


SyntaxError: 'return' outside function (2094296260.py, line 1)

In [34]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")
agent_executor = create_react_agent(model, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}} #need a config when execute so agent knows where to start

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")




{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 85, 'total_tokens': 96}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8febb757-f378-4198-af6c-0fd7d105e51b-0', usage_metadata={'input_tokens': 85, 'output_tokens': 11, 'total_tokens': 96})]}}
----


In [35]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")



{'agent': {'messages': [AIMessage(content='Your name is Bob!', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 108, 'total_tokens': 114}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-502363c3-f005-4e4d-8e8f-4b9205914e78-0', usage_metadata={'input_tokens': 108, 'output_tokens': 6, 'total_tokens': 114})]}}
----


[langgraph](https://langchain-ai.github.io/langgraph/): more information on langgraph, the main package for building agents